In [7]:
#%% [markdown]
# # Image Augmentation
#
# In this first section, we define functions to augment images.
# The augmentation techniques include random rotation, horizontal flip, random cropping, brightness and contrast adjustments, and scaling.
#
# After running these cells, we can generate augmented images which will be saved into a new folder.

#%% [code]
import os
import random
from PIL import Image, ImageEnhance

def augment_image(image):
    # random rotation between -15 and 15 degrees
    angle = random.randint(-15, 15)
    image = image.rotate(angle, expand=True)

    # horizontal flip
    if random.random() > 0.5:
        image = image.transpose(Image.FLIP_LEFT_RIGHT)

    # random cropping
    width, height = image.size
    new_width = int(width * random.uniform(0.8, 1.0))  # Crop width between 80% and 100%
    new_height = int(height * random.uniform(0.8, 1.0))  # Crop height between 80% and 100%

    # random position for cropping
    left = random.randint(0, width - new_width)
    top = random.randint(0, height - new_height)
    right = left + new_width
    bottom = top + new_height

    image = image.crop((left, top, right, bottom))

    # random brightness
    enhancer = ImageEnhance.Brightness(image)
    image = enhancer.enhance(random.uniform(0.5, 1.5))

    # contrast adjustment
    contrast_factors = [0.8, 1.0, 1.2]  # Lower, Normal, Higher
    contrast_factor = random.choice(contrast_factors)
    image = ImageEnhance.Contrast(image).enhance(contrast_factor)

    # scaling
    scale_factors = [0.8, 1.0, 1.2]  # Smaller, Normal, Larger
    scale_factor = random.choice(scale_factors)
    new_size = (int(image.width * scale_factor), int(image.height * scale_factor))
    image = image.resize(new_size, Image.ANTIALIAS)

    return image

def save_augmented_images(folder_path, output_folder, num_augmented_images=5):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    images = [f for f in os.listdir(folder_path) if f.lower().endswith('.jpg')]

    for img_name in images:
        img_path = os.path.join(folder_path, img_name)
        img = Image.open(img_path)
        for i in range(num_augmented_images):
            augmented_img = augment_image(img)
            augmented_img_name = f"{os.path.splitext(img_name)[0]}_augmented_{i}.jpg"
            augmented_img.save(os.path.join(output_folder, augmented_img_name))

#%% [markdown]
# **Example usage:**
#
# Uncomment and modify the following lines if you want to generate augmented images.
#
# ```python
# folder_path = 'DATASET/TRAIN/downdog'  # Path to the folder with original images
# output_folder = 'DATASET/TRAIN/downdog2'  # Path to save augmented images
# save_augmented_images(folder_path, output_folder, num_augmented_images=2)
# ```

#%% [markdown]
# # Data Preprocessing for Deep Learning
#
# This section loads the (augmented) images from your dataset, preprocesses them by resizing and normalizing, and produces:
# - **X**: the input matrix (images as numerical arrays)
# - **Y**: the output matrix (integer labels for each image)
#
# Optionally, we also provide one-hot encoded labels.

#%% [code]
import numpy as np
from PIL import Image

# Define the target image size (height, width)
IMG_SIZE = (128, 128)

# Path to your dataset folder (modify as needed)
dataset_path = '/Users/melitamadhurza/Desktop/AIT_Deep_Learning/Dataset/TRAIN'


# Initialize lists to hold the image data and labels
X = []
Y = []
class_names = []

# Loop through each subfolder (each subfolder represents a class)
for idx, folder in enumerate(sorted(os.listdir(dataset_path))):
    folder_path = os.path.join(dataset_path, folder)
    if os.path.isdir(folder_path):
        print(f"Processing class '{folder}' with label {idx}")
        class_names.append(folder)
        for filename in os.listdir(folder_path):
            if filename.lower().endswith('.jpg'):
                file_path = os.path.join(folder_path, filename)
                try:
                    # Open image, convert to RGB, and resize
                    img = Image.open(file_path).convert('RGB')
                    img = img.resize(IMG_SIZE)

                    # Convert image to numpy array and normalize pixel values to [0, 1]
                    img_array = np.array(img) / 255.0
                    X.append(img_array)

                    # Append label corresponding to the class index
                    Y.append(idx)
                except Exception as e:
                    print(f"Error processing {file_path}: {e}")

# Convert lists to numpy arrays
X = np.array(X)
Y = np.array(Y)

print("Shape of X (inputs):", X.shape)  # Expected: (number_of_images, 128, 128, 3)
print("Shape of Y (labels):", Y.shape)  # Expected: (number_of_images,)

#%% [markdown]
# Optionally, if you need the labels in one-hot encoding (e.g., for a classification network), use the following cell.

#%% [code]
from tensorflow.keras.utils import to_categorical

# Convert labels to one-hot encoded format
Y_onehot = to_categorical(Y, num_classes=len(class_names))
print("Shape of Y (one-hot encoded):", Y_onehot.shape)

#%% [markdown]
# Finally, save the matrices to disk for later use if desired.

#%% [code]
np.save('X.npy', X)
np.save('Y.npy', Y)
np.save('Y_onehot.npy', Y_onehot)

print("Data matrices saved successfully.")








FileNotFoundError: [Errno 2] No such file or directory: '/Users/melitamadhurza/Desktop/AIT_Deep_Learning/Dataset/TRAIN'